<a href="https://colab.research.google.com/github/redfrog66/NLP_Fake_News_Detection/blob/main/NLP_Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake news detection using NLP

Load the dataset:

In [78]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df_train = pd.read_parquet("hf://datasets/GonzaloA/fake_news/" + splits["train"])
df_validation = pd.read_parquet("hf://datasets/GonzaloA/fake_news/" + splits["validation"])
df_test = pd.read_parquet("hf://datasets/GonzaloA/fake_news/" + splits["test"])

df = pd.concat([df_train,df_validation,df_test])

In [84]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 40587 entries, 0 to 8116
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40587 non-null  object
 1   label   40587 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 951.3+ KB


,text,label
0,maury is perhaps one of the trashiest shows on...,0
1,yesterday after the father of one of the ucla...,0
2,moscow reuters russia on wednesday warned ...,1
3,house majority whip steve scalise r la th...,0
4,it can be said that late show host stephen col...,0


Drop unwanted properties

In [80]:
df = df.drop(['Unnamed: 0', 'title'], axis=1)

Text preprocessing

In [81]:
import re
import string
def preprocess_text(text):
  text = text.lower()
  text = re.sub(r'\[.*?\]|\W|https?://\S+|www\.\S+|<.*?>+|\n|\w*\d\w*', ' ', text)
  return text
df['text'] = df['text'].fillna('').apply(preprocess_text)

Delete empty rows

In [85]:
empty_rows_count = (df['text'].str.strip() == '').sum()
empty_rows_count

np.int64(57)

In [86]:
df = df[df['text'].str.strip().astype(bool)]

In [87]:
df.head()

,text,label
0,maury is perhaps one of the trashiest shows on...,0
1,yesterday after the father of one of the ucla...,0
2,moscow reuters russia on wednesday warned ...,1
3,house majority whip steve scalise r la th...,0
4,it can be said that late show host stephen col...,0


Split data to train and test

In [88]:
from sklearn.model_selection import train_test_split
x = df['text']
y = df['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20)

## Logistic regression:

Vectorize the data

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [90]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(xv_train,y_train)
pred_lr = lr.predict(xv_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_lr))
from sklearn.metrics import accuracy_score
print('Model accuracy:')
accuracy_score(y_test, pred_lr)

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3697
           1       0.97      0.97      0.97      4409

    accuracy                           0.97      8106
   macro avg       0.97      0.97      0.97      8106
weighted avg       0.97      0.97      0.97      8106

Model accuracy:


0.9676782630150506

TODO: Preprocess data differently(?), apply more models(BERT, LoRa, NN(?)), Add magic